# Lab 1: Short-Term Memory Agent

In Lab 0, the agent had no context and forgot everything after each turn.

In Lab 1, we add short-term memory. The agent now remembers the entire conversation within a single session using checkpointing and message accumulation. All messages are stored in the messages list, and each thread_id maintains its own conversation history.

However, this agent has a critical limitation: it has no episodic memory. While it remembers everything during a conversation, it forgets everything when you start a new session with a different thread_id. There is no mechanism to extract and store important facts that should persist across sessions or days.

This means the agent cannot answer questions like "What did we discuss yesterday?" because yesterday's session is gone. Additionally, as conversations grow longer, all messages are sent to the LLM every turn, causing prompt bloat.

In Lab 2, we will add episodic memory to solve these limitations by extracting, storing, and retrieving important facts across sessions.


In [ ]:
#!pip install langgraph langchain-openai langchain-core


In [4]:
from typing import Annotated, TypedDict, Optional
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_core.tools import tool
from langgraph.checkpoint.memory import InMemorySaver
import json

llm = ChatOpenAI(model="gpt-5.1")   


C:\Users\varsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
ticket_db = json.load(open("data/tickets.json"))

In [15]:
@tool
def lookup_ticket(ticket_id: str):
    """Lookup a ticket by ID."""
    ticket = ticket_db.get(ticket_id)
    if ticket is None:
        return {"error": f"Ticket {ticket_id} not found."}
    return ticket


@tool
def create_ticket(customer_name: str, issue: str, device: str = "-", priority: str = "Medium"):
    """Create a new ticket."""
    new_id = str(max(int(k) for k in ticket_db.keys()) + 1)

    ticket_db[new_id] = {
        "status": "New",
        "issue": issue,
        "description": issue,
        "device": device,
        "priority": priority,
        "created_at": "2025-12-12",
        "last_updated": "2025-12-12",
        "customer_name": customer_name,
        "notes": [
            {"timestamp": "2025-12-12T13:20:58", "author": "customer", "text": issue}
        ]
    }
    return {"ticket_id": new_id, "status": "created"}


TOOLS = [lookup_ticket, create_ticket]


In [16]:
class AgentState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]
    tool_result: Optional[dict]


In [ ]:
def planner_node(state: AgentState):

    latest = state["messages"][-1].content

    planning_instruction = """
    You are a CRM support planner.

    RULES:
    1. If the user provides a ticket ID (a 5–10 digit number) or asks for a ticket update/status/details/history, ALWAYS call:

    {
      "action": "tool",
      "tool": "lookup_ticket",
      "arguments": { "ticket_id": "..." },
      "response": ""
    }

    2. Only call create_ticket if the user clearly describes a NEW ISSUE and provides the REQUIRED fields:
      - customer_name
      - issue

    If you DO call create_ticket, arguments must strictly be:

    {
      "customer_name": "...",
      "issue": "...",
      "device": "-",
      "priority": "Medium"
    }

    3. If required fields are missing, DO NOT CALL create_ticket.
      Instead answer directly and ask the user for missing details.

    4. Never guess arguments. Never invent fields that the tool does not accept.

    Return strictly JSON:
    {
      "action": "tool" | "respond",
      "tool": "lookup_ticket" | "create_ticket" | "",
      "arguments": {},
      "response": ""
    }
    """

    messages = state["messages"] + [
        HumanMessage(content=planning_instruction)
    ]

    out = llm.invoke(messages)
    return {"messages": [AIMessage(content=out.content)]}


In [18]:
def tool_node(state: AgentState):
    raw = state["messages"][-1].content

    try:
        plan = json.loads(raw)
    except:
        return {"tool_result": {"error": "Invalid JSON from planner"}}

    if plan.get("action") == "tool":
        tool_name = plan["tool"]
        args = plan["arguments"]

        for t in TOOLS:
            if t.name == tool_name:
                return {"tool_result": t.run(args)}

        return {"tool_result": {"error": "Unknown tool"}}

    return {"tool_result": None}


In [ ]:
def response_node(state: AgentState):
    tool_result = state["tool_result"]

    response_instruction = f"""
    You are a stateless CRM support assistant.

    RULES:
    - You NEVER remember anything beyond what is present in the message history or the tool_result.
    - If tool_result contains a valid ticket (status, issue, etc.) → summarize it concisely.
    - If tool_result contains an error → ask the user for the ticket ID.
    - If the user mentions “ticket”, “reported”, “yesterday”, or “status” BUT no tool_result is provided → ask for the ticket ID (do NOT give troubleshooting).
    - Keep troubleshooting step short and clear.
    - Keep answers short, clear, and grounded only in the tool_result or user's message.
    - Never invent ticket fields or pretend to know past context.

    TOOL RESULT:
    {tool_result}

    Respond clearly and concisely to the user.
    """

    messages = state["messages"] + [
        HumanMessage(content=response_instruction)
    ]

    out = llm.invoke(messages)

    return {"messages": [out]}


In [20]:
checkpointer = InMemorySaver()
graph = StateGraph(AgentState)

graph.add_node("planner", planner_node)
graph.add_node("tool", tool_node)
graph.add_node("respond", response_node)

graph.set_entry_point("planner")
graph.add_edge("planner", "tool")
graph.add_edge("tool", "respond")
graph.add_edge("respond", END)

app = graph.compile(checkpointer=checkpointer)

In [24]:
from datetime import datetime

def pretty_print(role, text):
    print("\n" + "-" * 60)
    if role == "USER":
        print("🧑USER:")
        print(text)
    elif role == "AGENT":
        print("🤖 AGENT:")
        print(text)

def chat_loop(messages, thread_id="default_thread"):
    """
    messages: list of user strings
    thread_id: a persistent thread identifier for checkpointing
    """
    for user_text in messages:
        # invoke with only the new user message
        out = app.invoke(
            {"messages": [HumanMessage(content=user_text)]},
            {"configurable": {"thread_id": thread_id}}
        )

        # print user and agent
        pretty_print("USER", user_text)
        reply = out["messages"][-1].content
        pretty_print("AGENT", reply)

    print("\n" + "=" * 60)


In [25]:
input_sequence_day1 = [
    "My internet keeps dropping every night after 10 PM.",
    "I have already reported this issue yesterday.",
    "The ticket ID is 998878.",
    "Okay what's the latest status?",
    "Remind me what issue I originally reported."
]

print(">>> Session Day 1")
chat_loop(input_sequence_day1, thread_id="varsha_session")

print("\n----- RESET SESSION (fresh state, same thread) -----\n")

input_sequence_day2 = [
    "Can you check my ticket status again?",
    "Remind me what issue I reported yesterday.",
    "What troubleshooting steps did you give yesterday?"
]

print(">>> Session Day 2") #new chat session
chat_loop(input_sequence_day2, thread_id="varsha_session_new")


>>> Session Day 1

------------------------------------------------------------
🧑USER:
My internet keeps dropping every night after 10 PM.

------------------------------------------------------------
🤖 AGENT:
Your internet dropping every night after 10 PM could be caused by a Wi‑Fi schedule, parental controls, or a line issue.

Try these quick checks:
1. Restart your modem/router once before 10 PM.  
2. When it drops, check:
   - Do devices show “connected, no internet,” or does the Wi‑Fi name disappear?
   - Do any modem/router lights (especially Internet/WAN) turn red/off or blink differently?

Tell me what you see, and I’ll suggest the next step.

------------------------------------------------------------
🧑USER:
I have already reported this issue yesterday.

------------------------------------------------------------
🤖 AGENT:
You mentioned you already reported this, but I don’t have any ticket details in the tool result.

Please provide your **ticket ID** so I can check and summ